In [1]:
import sys 
sys.path.append('SGG_Benchmark_main')
from SGG_Benchmark_main.demo.demo_model import SGG_Model
import cv2
import os 

def get_pre(source):

    config_path = "output_motifs/config.yml"
    dict_path = "VG_dicts.json"
    weights_path = "output_motifs/best_model_epoch_4.pth"

    example_img = source  

    img = cv2.imread(example_img)
    img=cv2.resize(img,(1024,1024))

    model = SGG_Model(config_path, dict_path, weights_path, rel_conf=0.01, box_conf=0.1, show_fps=False)

    predictions = model.predict(img)

    return predictions

c:\ProgramData\Anaconda3\envs\RAG_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_box_pos(bbox,orig_size=[1024,1024]):
    """
    inputs:
        bbox 框的位置 xywh
        orig_size 图像大小 x,y heng shu
    outputs:
        框所在图片位置
        左上角，上部，右上角
        左部，  中部，右部
        左下角  下部  右下角
        012
        345
        678
           
    """
    center=((bbox[0]+bbox[2])/2,(bbox[1]+bbox[3])/2)    ##获取中心坐标
    #center=(bbox[0],bbox[1])

    if center[0]<orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top left corner'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top'
    if center[0]>2*orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top right corner'
    if center[0]<orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'left'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'center'
    if center[0]>2*orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'right'
    if center[0]<orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower left corner'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower'
    if center[0]>2*orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower right corner'



def deal_predictions(predictions):
    pos_obj_rel_pos_sub=[]         ## 位置object 关系 位置subject


    for i in range(len(predictions['rel_labels'])):
        obj_index,sub_index=predictions['rel_pairs'][i][0],predictions['rel_pairs'][i][1]

        obj,sub=predictions['bbox_labels'][obj_index],predictions['bbox_labels'][sub_index]
        obj_bbox,sub_bbox=predictions['bbox'][obj_index],predictions['bbox'][sub_index]
        rel=predictions['rel_labels'][i]

        pos_obj_rel_pos_sub.append([get_box_pos(obj_bbox),obj,rel,get_box_pos(sub_bbox),sub])

    object_count={}
    for i in predictions['bbox_labels']:
        if i in object_count.keys():
            object_count[i]+=1
        else:
            object_count[i]=1
    
    
    return pos_obj_rel_pos_sub,object_count

def fix(predictions,probs=0.15):
    fix=np.array(predictions['bbox_scores'])>probs

    bbox=np.array(predictions['bbox'])[fix]
    bbox_labels=np.array(predictions['bbox_labels'])[fix]

    bbox_idx=bbox_labels.shape[0]

    rel_pairs=[]
    rel_labels=[]

    for i,pair in enumerate(predictions['rel_pairs'][:50]):
        if pair[0]<bbox_idx and pair[1]<bbox_idx:
            rel_pairs.append(pair)
            rel_labels.append(predictions['rel_labels'][i])

    return list(bbox),list(bbox_labels),rel_pairs,rel_labels

def deal_SGG_result(predictions):


    bbox,bbox_labels,rel_pairs,rel_labels=fix(predictions)

    
    object_count={}
    #for i in predictions['bbox_labels']:
    for i in bbox_labels:
        if i in object_count.keys():
            object_count[i]+=1
        else:
            object_count[i]=1

    location_dict={key:[] for key in object_count.keys()}

    #for i in range(len(predictions['bbox'])):
    for i in range(len(bbox)):
        #location_dict[predictions['bbox_labels'][i]].append(get_box_pos(predictions['bbox'][i],shape))
        location_dict[bbox_labels[i]].append(get_box_pos(bbox[i]))


    relation_dict=[]

    #for i in range(len(predictions['rel_pairs'])):
    #    relation_dict.append([predictions['bbox_labels'][predictions['rel_pairs'][i][0]],predictions['rel_labels'][i],predictions['bbox_labels'][predictions['rel_pairs'][i][1]]])
    for i in range(len(rel_pairs)):
        relation_dict.append([bbox_labels[rel_pairs[i][0]],rel_labels[i],bbox_labels[rel_pairs[i][1]]])

    return {"number":object_count,"location":location_dict,"relationship":relation_dict}



    

In [6]:
from openai import OpenAI
import json
import os


In [269]:
def get_response(pre):
    client = OpenAI(
        api_key="Your keys", 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  #
    )

    Question='Please complete the following task based on the data {} (which contains three parts of information: quantity,location,and relationship):'.format(json.dumps(pre))+'{"Task": "Please provide me with the quantity, position, and relationship of all different objects in the given image in JSON format,only output json\
    For quantity, count all the quantities of different objects. If it is 0, do not output that class,\
    For the positions of different objects, the position of each individual unit in each type of object needs to be output(the positions are divided into nine grids, namely the top left corner),\
    top,top right corner,left,center,right,left lower corner,lower,right lower corner)。\
    For relationships, output the relationships between all objects in the image in the form of triplets, [object1, relation,object2],\
    Please refer to the following JSON format for specific formatting, and use this JSON format for output (just output one JSON)",\
    "Restriction": "Strictly provide JSON responses in <format> format,Do not output any other information and irrelevant symbols,Please provide me with a string format and do not output the syntax for markdown",\
    "format":{"number":{"man":2,"car":2,"woman":1},"location":{"man":{1:"center",2:"lower"},"car":{1,:"top right corner",2:"left"},"woman":{1:"center"} },"relationship":[["man","near to","car"],["car","on","load"],["woman","has","car"]]} }'


    completion = client.chat.completions.create(
        model="qwen-plus-0806",
        messages=[
            {'role': 'user', 'content': Question}],
        seed=2
        )
    return completion

In [3]:
import numpy as np
pre_output_list=[]

list_dir=os.listdir('JPEGImages')

import pickle 

for i in list_dir[:100]:

    predictions=get_pre(os.path.join('JPEGImages',i))

    pre=deal_SGG_result(predictions)

    #pre_output_list.append(pre)

    with open(os.path.join('pre/'+i[:3]+'.pickle'),'wb') as f:
        pickle.dump(pre,f)

2024-09-06 18:54:59.939 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:30 - ----------------------------------------------------------------------------------------------------
2024-09-06 18:54:59.939 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:31 - get dataset statistics...
2024-09-06 18:54:59.941 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:42 - Loading data statistics from: C:\Users\Administrator\Desktop\RAG\out\VG150_train_statistics.cache
2024-09-06 18:54:59.941 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:43 - ----------------------------------------------------------------------------------------------------


loading word vectors from .\glove.6B.200d.pt
loading word vectors from .\glove.6B.200d.pt


2024-09-06 18:55:02.865 | INFO     | sgg_benchmark.utils.checkpoint:load:65 - Loading checkpoint from output_motifs/best_model_epoch_4.pth
c:\ProgramData\Anaconda3\envs\RAG_test\lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2024-09-06 18:55:06.767 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:30 - ----------------------------------------------------------------------------------------------------
2024-09-06 18:55:06.768 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:31 - get dataset statistics...
2024-09-06 18:55:06.769 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:42 - Loading data statistics from: C:\Users\Administrator\Desktop\RAG\out\VG150_train_statistics.cache
2024-09

loading word vectors from .\glove.6B.200d.pt
loading word vectors from .\glove.6B.200d.pt


2024-09-06 18:55:09.496 | INFO     | sgg_benchmark.utils.checkpoint:load:65 - Loading checkpoint from output_motifs/best_model_epoch_4.pth
2024-09-06 18:55:13.022 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:30 - ----------------------------------------------------------------------------------------------------
2024-09-06 18:55:13.022 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:31 - get dataset statistics...
2024-09-06 18:55:13.024 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:42 - Loading data statistics from: C:\Users\Administrator\Desktop\RAG\out\VG150_train_statistics.cache
2024-09-06 18:55:13.024 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:43 - ----------------------------------------------------------------------------------------------------


...

2024-09-06 19:05:40.087 | INFO     | sgg_benchmark.utils.checkpoint:load:65 - Loading checkpoint from output_motifs/best_model_epoch_4.pth
2024-09-06 19:05:43.495 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:30 - ----------------------------------------------------------------------------------------------------
2024-09-06 19:05:43.496 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:31 - get dataset statistics...
2024-09-06 19:05:43.497 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:42 - Loading data statistics from: C:\Users\Administrator\Desktop\RAG\out\VG150_train_statistics.cache
2024-09-06 19:05:43.498 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:43 - ----------------------------------------------------------------------------------------------------


loading word vectors from .\glove.6B.200d.pt
loading word vectors from .\glove.6B.200d.pt


2024-09-06 19:05:46.437 | INFO     | sgg_benchmark.utils.checkpoint:load:65 - Loading checkpoint from output_motifs/best_model_epoch_4.pth
2024-09-06 19:05:50.076 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:30 - ----------------------------------------------------------------------------------------------------
2024-09-06 19:05:50.077 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:31 - get dataset statistics...
2024-09-06 19:05:50.078 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:42 - Loading data statistics from: C:\Users\Administrator\Desktop\RAG\out\VG150_train_statistics.cache
2024-09-06 19:05:50.078 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:43 - ----------------------------------------------------------------------------------------------------


loading word vectors from .\glove.6B.200d.pt
loading word vectors from .\glove.6B.200d.pt


2024-09-06 19:05:53.029 | INFO     | sgg_benchmark.utils.checkpoint:load:65 - Loading checkpoint from output_motifs/best_model_epoch_4.pth


In [300]:
with open(os.path.join('my/596.pickle'),'wb') as f:
    pickle.dump(out,f)

In [ ]:
list_dir=os.listdir('pre')

import pickle 


for j,i in enumerate(list_dir[-100:]):


    with open('pre/'+i,'rb') as f:
        pre=pickle.load(f)


    output=get_response(pre)

    out=output.choices[0].message.content

    predict=demjson3.decode(out)

    with open(os.path.join('my',i),'wb') as f:
        pickle.dump(out,f)

    print(i)

In [4]:
import pickle
import os 
import nltk
import demjson3
from nltk.corpus import wordnet
import re

In [5]:
def are_synonyms_split(word_list,word2):
    """第一个是一个列表"""
    right=[]

    for i in word_list:
        for j in word2:
            right.append(are_synonyms(i,j))

    if True in right:
        return True 
    else:
        return False

def has_hypernym_or_hyponym_split(word_list,word2):
    """第一个是一个列表"""
    right=[]

    for i in word_list:
        for j in word2:
            right.append(has_hypernym_or_hyponym(i,j))

    if True in right:
        return True 
    else:
        return False

def are_synonyms(word1, word2):
    # 获取word1的所有同义词集
    synsets1 = wordnet.synsets(word1)
    
    # 获取word2的所有同义词集
    synsets2 = wordnet.synsets(word2)

    
    
    # 检查是否有交集
    for synset1 in synsets1:
        for synset2 in synsets2:
            if synset1 == synset2:
                return True
    
    # 计算路径相似度作为备选方案
    for synset1 in synsets1:
        for synset2 in synsets2:
            if synset1.path_similarity(synset2) is not None and synset1.path_similarity(synset2) > 0.8:
                return True
    
    return False

def has_hypernym_or_hyponym(word1, word2):
    # 获取word1的所有同义词集
    synsets1 = wordnet.synsets(word1)
    
    # 获取word2的所有同义词集
    synsets2 = wordnet.synsets(word2)
    
    # 检查是否存在上位词或下位词关系
    for synset1 in synsets1:
        for synset2 in synsets2:
            # 检查synset1是否是synset2的上位词
            if synset1 in synset2.hypernyms():
                return True
            
            # 检查synset2是否是synset1的上位词
            if synset2 in synset1.hypernyms():
                return True
    
    return False

def count_common_elements_with_repetition(list1, list2):
    # 创建字典来存储每个元素在两个列表中的出现次数
    count_dict1 = {}
    count_dict2 = {}
    
    # 统计第一个列表中每个元素的出现次数
    for element in list1:
        if element in count_dict1:
            count_dict1[element] += 1
        else:
            count_dict1[element] = 1
    
    # 统计第二个列表中每个元素的出现次数
    for element in list2:
        if element in count_dict2:
            count_dict2[element] += 1
        else:
            count_dict2[element] = 1
    
    # 计算共同元素的数量
    common_count = 0
    for element in count_dict1:
        if element in count_dict2:
            # 取两个列表中该元素出现次数的较小值
            common_count += min(count_dict1[element], count_dict2[element])
    
    return common_count

In [6]:
def compute_right_single(ground_truth,predict):
    """
        计算单张图片的预测准确率
        分成四个 计数，位置，关系，total
    """
    
    category_right=0
    number_right=0
    loc_right=0

    ready_list=[]

    for i in predict['number'].keys():
        for j in ground_truth['number'].keys():
            if j in ready_list:
                continue
            if i==j:        ##标签对应上了
                category_right+=1
                ready_list.append(j)

                if predict['number'][i]==ground_truth['number'][j]:     #统计数量一样，加1
                    number_right+=1
                ###计算位置对了几个
                loc_right+=count_common_elements_with_repetition(predict['location'][i],ground_truth['location'][j])
                break

    rel_right=0        ###计算关系预测对了几个   逻辑上 关系三元组，一个个去比

  
    rel_ready=[]

    for i in predict['relationship']:
        for k,j in enumerate(ground_truth['relationship']):
            if k in rel_ready:
                continue
            if i[0]==j[0] and i[1]==j[1] and i[2]==j[2]:
                rel_ready.append(k)
                rel_right+=1     
                break


    ### 计算ground_truth总数
    number_num=len(ground_truth['number'].keys())
    loc_num=sum(ground_truth['number'].values())
    rel_num=len(ground_truth['relationship'])

    ### 计算生成结果的总数
    gen_num=len(predict['number'].keys())
    gen_loc=sum(predict['number'].values())
    gen_rel=len(predict['relationship'])

    ###计算 precision

    if gen_num==0:
        number_precision=0
        category_precision=0
    else:
        category_precision=category_right/gen_num
        number_precision=number_right/gen_num
    if gen_loc==0:
        loc_precision=0
    else:
        loc_precision=loc_right/gen_loc
    if gen_rel==0:
        rel_precision=0
    else:
        rel_precision=rel_right/gen_rel

    ### 计算recall
    if number_num==0:
        number_recall=1
        category_recall=1
    else:
        category_recall=category_right/number_num
        number_recall=number_right/number_num

    if loc_num==0:
        loc_recall=1
    else:
        loc_recall=loc_right/loc_num
    
    if rel_num==0:
        rel_recall=1
    else:
        rel_recall=rel_right/rel_num

    #### 计算F1
    if category_precision==0 and category_recall==0:
        category_f1=0
    else:
        category_f1=2*(category_precision*category_recall)/(category_precision+category_recall)
    if number_precision==0 and number_recall==0:
        num_f1=0
    else:
        num_f1=2*(number_precision*number_recall)/(number_precision+number_recall)
    
    if loc_precision==0 and loc_recall==0:
        loc_f1=0
    else:
        loc_f1=2*(loc_precision*loc_recall)/(loc_precision+loc_recall)
    
    if rel_precision==0 and rel_recall==0:
        rel_f1=0
    else:
        rel_f1=2*(rel_precision*rel_recall)/(rel_precision+rel_recall)

    return category_precision,number_precision,loc_precision,rel_precision,category_recall,number_recall,loc_recall,rel_recall,category_f1,num_f1,loc_f1,rel_f1


In [7]:
with open('ground_truth.pickle','rb') as f:
    ground_truth_total=pickle.load(f)

In [8]:
acc_list=[]

answer_list=os.listdir('pre')

for i,answer in enumerate(answer_list):

    with open(os.path.join('pre',answer),'rb') as f:
        data=pickle.load(f)
        #print(answer)
    #predict=demjson3.decode(data)
    ground_truth=ground_truth_total[i]
    print(len(data['relationship']))

    acc_list.append(list(compute_right_single(ground_truth,data)))

50
50
50
46
50
16
45
43
48
42
37
50
50
50
49
37
50
36
50
38
46
43
49
50
45
50
50
41
46
44
48
50
50
50
50
50
50
38
41
50
42
50
2
47
47
50
50
40
48
47
31
39
37
34
48
23
46
49
50
48
8
22
48
50
45
50
43
50
50
26
23
50
50
49
42
11
46
47
31
36
36
28
47
50
49
42
50
44
49
32
41
49
50
16
50
44
1
50
15
50


In [9]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)
acc_numpy=np.array(acc_list)
acc_numpy.mean(axis=0)

array([0.8317, 0.3786, 0.556 , 0.1033, 0.6435, 0.3048, 0.332 , 0.164 , 0.7078, 0.3301, 0.3971, 0.1169])

In [10]:
(acc_numpy>0.55)[:,4:8].mean(1).mean()

0.225